<h1 align="center">Assembling or loading anndata object</h1> 
<hr style="border:2px solid black"> </hr>

# A: set up

In [1]:
#####################IMPORTING PACKAGES################################
import sctoolbox
from sctoolbox.checker import *
from sctoolbox.creators import *
from sctoolbox.file_converter import *
from sctoolbox.assemblers import *

#####################DEFINING TEST NUMBER##############################
test="Test1"

#####################DEFINING STRATEGY#################################
assembling_10_velocity=True #cells C:
assembling_10_public=False #cells D:
convert_seurat=False #cells E:

######################OUTPUT PATH######################################
path_out="/mnt/workspace/gvalent/5_SC_automatation/data2" #The path where the folders results/Test* will be created to export your results

##########################DEFINING IMPUTS##############################
#####10X assembling for velocity (cell C:)
path_solo_quant="/mnt/agnerds/loosolab_SC_RNA_framework/examples/assembling_10_velocity/quant/" #This is the directory where the quant folder from snakemake preprocessing is located
                #NOTE: insert the pathway including the /quant folder
                #e.g., /path/quant 

#####Converting from seurat to anndata (cell E:)
path_rds="/mnt/agnerds/loosolab_SC_RNA_framework/examples/convert_seurat/GSE135893_ILD_annotated_fullsize.rds" #This is the directory with the Seurat (.rds) file

#####10X assembling from public data (cell D:)
#See set up in cell D

# B: checking and creating the output directory

In [2]:
#Creating directories to store the outputs and storing this information
output_path(path_out, test)

Output directory is ready: /mnt/workspace/gvalent/5_SC_automatation/data2/results/Test1/


# C: assembling 10X anndata for velocity

In [3]:
#Assembling an anndata object for velocity analysis
#Set up the information bellow
the_10X_yml=[    
"sample1:condition:room_air",
"sample2:condition:smooke_exposure",
]

##########################################################################
if assembling_10_velocity == True:
    adata=assembling_velocity(path_solo_quant, the_10X_yml, assembling_10_velocity, test)
    display(adata)

Num samples: 2
sample1:condition:room_air
Assembling sample 1
		Loading matrix to compose the .X object
		Loading matrix to compose the .obs
		Loading matrix to compose the .var


KeyboardInterrupt: 

# D: assembling an anndata object from 10X public data

In [6]:
#Assembling an anndata object from 10X public data
#Set up the information bellow
mtx = ["path_mtx_file"] #Path to matrix file(s)
barcodes = ["path_barcode_file"] #Path to barcode file(s)
genes = ["path_genes_file"] #Path to genes file(s)

if assembling_10_public == True:
    adata=assemblers.from_mtx(mtx, barcodes, genes, **kwargs)
    display(adata)

# E: converting from Seurat to anndata object

In [7]:
# Converting from Seurat to anndata object
if convert_seurat == True:
    adata = converter.convertToAdata(file=path_rds)
    display(adata)